<a href="https://colab.research.google.com/github/HongJaeKwon/kimhae_fire_detection/blob/master/hjk_fire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
# /gdrive/My Drive/ (폴더명)

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import random
from tqdm import tqdm
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import losses
from tensorflow.python.keras import initializers
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, Activation, Flatten, Dense, UpSampling2D, Reshape, Lambda, Input, Dropout, LeakyReLU, BatchNormalization
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, Iterator
from tensorflow.python.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.python.keras.losses import mean_absolute_error, binary_crossentropy

In [ ]:
train_df = pd.read_csv('/gdrive/My Drive/김해화재/PJT002_train(1).csv')
val_df = pd.read_csv('/gdrive/My Drive/김해화재/PJT002_validation(1).csv')
test_df= pd.read_csv('/gdrive/My Drive/김해화재/PJT002_test(1).csv')

In [ ]:
test_df['fr_yn']='U'

In [ ]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(59199, 173)
(6898, 173)
(2957, 173)


In [ ]:
all_df=pd.concat([train_df,val_df,test_df],axis=0)

In [ ]:
all_f=all_df.columns.to_list()
cat_f=all_df.columns.to_list()
num_f=all_df.describe().columns.to_list()

In [ ]:
for f in num_f:
    cat_f.remove(f)

In [ ]:
print(len(all_f))
print(len(cat_f))
print(len(num_f))
print(all_df.shape)

In [ ]:
all_df[num_f]=all_df[num_f].fillna(-1)

In [ ]:
all_df[cat_f]=all_df[cat_f].fillna("U")

In [ ]:
for f in cat_f:
    print(f,len(all_df[f].unique()))

In [ ]:
cat_f.remove('dt_of_athrztn')
cat_f.remove('dt_of_fr')
cat_f.remove('emd_nm')
cat_f.remove('fr_yn')

In [ ]:
for f in cat_f:
    print(f,len(all_df[f].unique()))

In [ ]:
one_hot=pd.get_dummies(all_df[cat_f])

In [ ]:
all_df= all_df.drop( cat_f , axis = 1)
all_df= all_df.drop('dt_of_athrztn' , axis = 1)
all_df= all_df.drop('dt_of_fr' , axis = 1)
all_df= all_df.drop('emd_nm' , axis = 1)

In [ ]:
all_df=pd.concat([all_df,one_hot],axis = 1)

In [ ]:
all_df['fr_yn']=all_df['fr_yn'].apply(lambda x: 0 if x=='N' else 1 if x=='Y' else -1 )

In [ ]:
all_f=all_df.columns.to_list()
all_f.remove('fr_yn')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
all_df[all_f] =scaler.fit_transform(all_df[all_f])

In [ ]:
all_y=all_df['fr_yn'].as_matrix()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
all_df_pca=pca.fit_transform(all_df[all_f]) 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
all_df_pca =scaler.fit_transform(all_df_pca)

In [ ]:
print(all_y.shape)
print(all_df_pca.shape)
print(pca.explained_variance_ratio_.sum())

(69054,)
(69054, 100)
0.9835242774972811


In [ ]:
all_y=np.reshape(all_y,(-1,1))

In [ ]:
all_DF=pd.DataFrame(all_df_pca,columns=list( "col_" + str(i) for i in range(100)))

In [ ]:
all_DF_Y=pd.DataFrame(all_y,columns=['LABEL'])

In [ ]:
all_DF=pd.concat([all_DF,all_DF_Y],axis = 1)

In [ ]:
train_DF=all_DF[:train_df.shape[0]]
val_DF=all_DF[train_df.shape[0]:train_df.shape[0]+val_df.shape[0]]
test_DF=all_DF[all_DF['LABEL']==-1]

train_DF_Y=train_DF['LABEL']
val_DF_Y=val_DF['LABEL']
train_DF_normal=train_DF[train_DF['LABEL']==0]
train_DF_fire=train_DF[train_DF['LABEL']==1]

train_DF=train_DF.drop('LABEL', axis = 1)
val_DF=val_DF.drop('LABEL', axis = 1)
test_DF=test_DF.drop('LABEL', axis = 1)

train_DF_normal=train_DF_normal.drop('LABEL', axis = 1)
train_DF_fire=train_DF_fire.drop('LABEL', axis = 1)


In [ ]:
f=train_DF_normal.columns.to_list()

In [ ]:
for fe in f:
    print (fe,"정상 평균 : ", train_DF_normal[fe].mean())
    print (fe,"정상 표준편차 : ", train_DF_normal[fe].std())
    print (fe,"화재 평균 : ", train_DF_fire[fe].mean())
    print (fe,"화재 표준편차 : ", train_DF_fire[fe].std())
    print ("")

col_0 정상 평균 :  0.5650823638378032
col_0 정상 표준편차 :  0.3745185390667692
col_0 화재 평균 :  0.2476339213245645
col_0 화재 표준편차 :  0.16384080250120894

col_1 정상 평균 :  0.3203549174211007
col_1 정상 표준편차 :  0.218230516334277
col_1 화재 평균 :  0.48072378015640804
col_1 화재 표준편차 :  0.2878127477984686

col_2 정상 평균 :  0.3704506025585312
col_2 정상 표준편차 :  0.17694438736114443
col_2 화재 평균 :  0.4268428443415771
col_2 화재 표준편차 :  0.19535047685043264

col_3 정상 평균 :  0.42725707585024014
col_3 정상 표준편차 :  0.1955373187186029
col_3 화재 평균 :  0.44675184614972213
col_3 화재 표준편차 :  0.18653603772342917

col_4 정상 평균 :  0.4541843163409643
col_4 정상 표준편차 :  0.14249426769478674
col_4 화재 평균 :  0.5110441679876078
col_4 화재 표준편차 :  0.17316059094543018

col_5 정상 평균 :  0.47477877674722485
col_5 정상 표준편차 :  0.16191619856571893
col_5 화재 평균 :  0.4537946296289703
col_5 화재 표준편차 :  0.177727524212021

col_6 정상 평균 :  0.4404964046561243
col_6 정상 표준편차 :  0.10513506744630359
col_6 화재 평균 :  0.4550196112612747
col_6 화재 표준편차 :  0.13178080417945526

co

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=5)
decision_tree = decision_tree.fit(train_DF, train_DF_Y)


In [ ]:
y_pred=decision_tree.predict(val_DF)
f1_score(val_DF_Y, y_pred)

0.20703653585926926

In [ ]:
from  sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=42)
rf_clf.fit(train_DF, train_DF_Y)
y_pred=rf_clf.predict(val_DF)
f1_score(val_DF_Y, y_pred)

0.40503035559410233

In [ ]:
class HJK_AE:
    def __init__(self,batch_size=128, input_size=100):
        self.batch_size=batch_size
        self.input_size=input_size
        self.lr = 0.001
        self.z_size=10
        
    def buld_model(self):
        inputs=Input(self.input_size,)
        encoder=self.build_encoder(self.input_size,self.z_size)  
        self.decoder=self.build_decoder(self.input_size,self.z_size)
              
        self.AE=Model(inputs=inputs,outputs=self.decoder(encoder(inputs))) 
        self.AE.compile(loss='mse', optimizer=Adam(self.lr))

        self.discriminator=self.build_discriminator(self.input_size,self.z_size)
        self.discriminator.compile(loss='binary_crossentropy',metrics=['acc',self.f1_m,self.precision_m, self.recall_m],optimizer=Adam(self.lr))

    def hjk_train_AE(self,trian_X,epochs=20):
        self.AE.fit(trian_X,trian_X,batch_size=self.batch_size,epochs=epochs)
    
    def hjk_train_discriminator(self,train_X,train_Y,epochs = 20):
        AE_train_X=self.AE.predict(train_X)
        self.discriminator.fit(AE_train_X,train_Y,validation_split=0.3, epochs=epochs)
            
    def hjk_predict(self,trian_X):
        return self.discriminator.predict(self.AE.predict(trian_X))
    
    def build_discriminator(self,input_size,z_size):
        model = Sequential()
        model.add(Dense(15,activation='relu',input_shape=(input_size,)))
        model.add(Dense(z_size,activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
        return model

    def build_encoder(self,input_size, z_size):
        model = Sequential()
        # model.add(Lambda((lambda x: x/16.),input_shape=(input_size,)))
        model.add(Dense(15,activation='relu'))
        model.add(Dense(z_size,activation='tanh'))
        return model
    
    def build_decoder(self,input_size, z_size):
        model = Sequential()
        model.add(Dense(15,activation='relu',input_shape=(z_size,)))
        model.add(Dense(input_size,activation='sigmoid'))
        # model.add(Lambda(lambda x: x * 16.))      
        return model
    
    def recall_m(self,y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(self,y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(self,y_true, y_pred):
        precision = self.precision_m(y_true, y_pred)
        recall = self.recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
hjk=HJK_AE()
hjk.buld_model()

In [ ]:
hjk.hjk_train_AE(train_DF_fire.as_matrix(),100)

In [ ]:
hjk.hjk_train_discriminator(train_DF.as_matrix(),train_DF_Y.as_matrix(),50)

In [ ]:
real_list=np.reshape(hjk.hjk_predict(train_DF.as_matrix()),(-1,1))
dt_list=np.reshape(decision_tree.predict_proba(train_DF)[:,1],(-1,1))
rf_list=np.reshape(rf_clf.predict_proba(train_DF)[:,1],(-1,1))
print(real_list.shape)
print(dt_list.shape)
print(rf_list.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(59199, 1)
(59199, 1)
(59199, 1)


In [ ]:
t_X=np.hstack((real_list,dt_list,rf_list))

In [ ]:
stack_model = Sequential()
stack_model.add(Dense(10,activation='relu',input_shape=(3,)))
stack_model.add(Dense(5,activation='relu'))
stack_model.add(Dense(1,activation='sigmoid'))
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def hjk_loss(y_true,y_pred):
    bce = 5*y_true * K.log(y_pred + K.epsilon())
    bce += (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    return -bce   

stack_model.compile(loss=hjk_loss,metrics=['acc',f1_m,precision_m, recall_m],optimizer=Adam(0.01))

In [ ]:
stack_model.fit(t_X,train_DF_Y.as_matrix(),batch_size=100,epochs=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Train on 59199 samples
Epoch 1/5
59199/59199 [==============================] - 2s 36us/sample - loss: 0.0564 - acc: 0.9933 - f1_m: 0.9858 - precision_m: 0.9794 - recall_m: 1.0000
Epoch 2/5
59199/59199 [==============================] - 2s 34us/sample - loss: 1.6249e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 3/5
59199/59199 [==============================] - 2s 35us/sample - loss: 5.9280e-05 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 4/5
59199/59199 [==============================] - 2s 34us/sample - loss: 3.1344e-05 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 5/5
59199/59199 [==============================] - 2s 34us/sample - loss: 1.9905e-05 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000


In [ ]:
real_list=np.reshape(hjk.hjk_predict(val_DF.as_matrix()),(-1,1))
dt_list=np.reshape(decision_tree.predict_proba(val_DF)[:,1],(-1,1))
rf_list=np.reshape(rf_clf.predict_proba(val_DF)[:,1],(-1,1))
print(real_list.shape)
print(dt_list.shape)
print(rf_list.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(6898, 1)
(6898, 1)
(6898, 1)


In [ ]:
v_X=np.hstack((real_list,dt_list,rf_list))

In [ ]:
stack_model.evaluate(v_X,val_DF_Y.as_matrix())

  32/6898 [..............................] - ETA: 13s - loss: 4.9505 - acc: 0.8125 - f1_m: 0.2500 - precision_m: 0.3333 - recall_m: 0.2000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


6898/6898 [==============================] - 1s 75us/sample - loss: 3.8536 - acc: 0.7802 - f1_m: 0.3832 - precision_m: 0.4106 - recall_m: 0.4042


[3.853619851800593, 0.7802262, 0.38317773, 0.41064316, 0.4041876]